# Kriging
######[Nomenclature](./nomenclature.html)


Kriging is an interpolating model that is a linear combination of a known function $f_i({\bf x})$ which is added to a realization of a stochastic process $Z({\bf x})$
\begin{equation}
\hat{y} = \sum\limits_{i=1}^k\beta_if_i({\bf x})+Z({\bf x}).
\end{equation}

$Z({\bf x})$ is a realization of a stochastique process with mean zero and spatial covariance function given by
\begin{equation}
cov\left[Z\left({\bf x}^{(i)}\right),Z\left({\bf x}^{(j)}\right)\right] =\sigma^2R\left({\bf x}^{(i)},{\bf x}^{(j)}\right)
\end{equation}

where $\sigma^2$ is the process variance, and $R$ is the correlation.
Two types of correlation functions are available in SMT: the exponential (Ornstein-Uhlenbeck process) and Gaussian correlation functions
\begin{equation}
\prod\limits_{l=1}^d\exp\left(-\theta_l\left|x_l^{(i)}-x_l^{(j)}\right|\right),\qquad \qquad \qquad\prod\limits_{l=1}^d\exp\left(-\theta_l\left(x_l^{(i)}-x_l^{(j)}\right)^{2}\right) \quad \forall\ \theta_l\in\mathbb{R}^+\\
\text{Exponential correlation function} \quad \qquad\text{Gaussian correlation function}\qquad \qquad
\end{equation}
These two correlation functions are called by 'abs_exp' (exponential) and 'squar_exp' (Gaussian) in SMT.

The deterministic term $\sum\limits_{i=1}^k\beta_i f_i({\bf x})$ can be replaced by a constant, a linear model, or a quadratic model.
These three types are available in SMT.

More details about the kriging approach could be found in [1].

[1] J. Sacks, S. B. Schiller, W. J. Welch, Designs for computer experiments, Technometrics 31 (1) (1989) 41--47.


## Options

| Option | Default | Acceptable values | Acceptable types | Description |
| - | - | - | - | - |
| theta0 | None | None | ['list', 'ndarray'] | Initial hyperparameters |
| data_dir | None | None | ['str'] | Directory for loading / saving cached data; None means do not save or load |
| print_solver | True | None | ['bool'] | Whether to print solver information |
| print_problem | True | None | ['bool'] | Whether to print problem information |
| print_global | True | None | ['bool'] | Global print toggle. If False, all printing is suppressed |
| poly | constant | ('constant', 'linear', 'quadratic') | ['function'] | regr. term |
| corr | squar_exp | ('abs_exp', 'squar_exp') | ['function'] | type of corr. func. |
| print_training | True | None | ['bool'] | Whether to print training information |
| print_prediction | True | None | ['bool'] | Whether to print prediction information |

## Example code

In [1]:
from __future__ import print_function, division
import numpy as np
from smt.methods.krg import KRG
from scipy import linalg
from smt.problems import Sphere
from smt.sampling import LHS


# Initialization of the problem
ndim = 10
ndoe = int(10*ndim)

# Define the function
fun = Sphere(ndim = ndim)

# Construction of the DOE
sampling = LHS(xlimits=fun.xlimits,criterion = 'm')
xt = sampling(ndoe)

# Compute the output
yt = fun(xt)
# Compute the gradient
for i in range(ndim):
    yd = fun(xt,kx=i)
    yt = np.concatenate((yt,yd),axis=1)

# Construction of the validation points
ntest = 500
sampling = LHS(xlimits=fun.xlimits)
xtest = sampling(ntest)
ytest = fun(xtest)

########### The Kriging model
# The variable 'theta0' is a list of length ndim.
t = KRG(theta0=[1e-2]*ndim)
t.set_training_values(xt,yt[:,0])

t.train()
y = t.predict_values(xtest)

print('Kriging,  err: '+str(linalg.norm(y.reshape((ntest,1))-ytest.reshape((ntest,
            1)))/linalg.norm(ytest.reshape((ntest,1)))))

___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 100
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  0.0933559
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 500
   
   Predicting ...
   Predicting - done. Time (sec):  0.0091372
   
   Prediction time/pt. (sec) :  0.0000183
   
Kriging,  err: 0.0105911717832
